In [27]:

import mlflow
import mlflow.sklearn
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
import boto3
import pandas as pd
from dotenv import load_dotenv
import os
from xgboost import XGBClassifier
from sklearn.model_selection import cross_val_score
import warnings
import numpy as np
from mlflow.models.signature import infer_signature
import urllib3
urllib3.disable_warnings(urllib3.exceptions.InsecureRequestWarning)

In [28]:
load_dotenv(dotenv_path="../.env")

access_key = os.getenv("AWS_ACCESS_KEY_ID_CLIENT")
secret_key = os.getenv("AWS_SECRET_ACCESS_KEY_CLIENT")
s3_url = os.getenv("MLFLOW_S3_ENDPOINT_URL_CLIENT")
tracker_url = os.getenv("MLFLOW_URL_CLIENT")
os.environ["MLFLOW_S3_ENDPOINT_URL"] = os.getenv("MLFLOW_S3_ENDPOINT_URL_CLIENT")
os.environ["MLFLOW_TRACKING_USERNAME"] = os.getenv("MLFLOW_ADMIN_USERNAME_CLIENT")
os.environ["MLFLOW_TRACKING_PASSWORD"] = os.getenv("MLFLOW_ADMIN_PASSWORD_CLIENT")
os.environ["MLFLOW_S3_IGNORE_TLS"] = "true"
os.environ["MLFLOW_ENABLE_SYSTEM_METRICS_LOGGING"] = "true"
bucket_name = "dataset"
object_name = "water_potability.csv"

In [29]:
print (secret_key)

magang123


In [30]:
s3 = boto3.client(
    's3',
    endpoint_url=s3_url,
    aws_access_key_id=access_key,
    aws_secret_access_key=secret_key,
    verify=False
)

In [31]:
try:
    # Attempt to list buckets
    response = s3.list_buckets()
    
    # If successful, print bucket names
    print("Connected successfully! Buckets available:")
    for bucket in response['Buckets']:
        print(f"- {bucket['Name']}")
except Exception as e:
    # Print any connection errors
    print("Connection error:", e)


Connected successfully! Buckets available:
- dataset
- mlflow


In [32]:
try:
    response = s3.get_object(Bucket=bucket_name, Key=object_name)
    print(response)
    dataset_content = response.get('Body')
    print(dataset_content)
    # Load the dataset into a DataFrame
    df = pd.read_csv(dataset_content)
    print("Dataset loaded successfully:")
    print(df.head())
except Exception as e:
    print("Error fetching dataset from MinIO:", e)


{'ResponseMetadata': {'RequestId': '181901C730E647F1', 'HostId': 'b675f5c02385af78c69266a96b22038bea3ecd7423814977a7267fd68ac2bbeb', 'HTTPStatusCode': 200, 'HTTPHeaders': {'accept-ranges': 'bytes', 'content-length': '525187', 'content-type': 'text/csv', 'etag': '"a5eeae83d792e40da6c2f9e1eab945fc"', 'last-modified': 'Thu, 09 Jan 2025 11:07:10 GMT', 'server': 'MinIO', 'strict-transport-security': 'max-age=31536000; includeSubDomains', 'vary': 'Origin, Accept-Encoding', 'x-amz-id-2': 'b675f5c02385af78c69266a96b22038bea3ecd7423814977a7267fd68ac2bbeb', 'x-amz-request-id': '181901C730E647F1', 'x-content-type-options': 'nosniff', 'x-ratelimit-limit': '2104', 'x-ratelimit-remaining': '2104', 'x-xss-protection': '1; mode=block', 'x-amz-version-id': 'c83a412d-03eb-40d4-aee5-4b48b61d6269', 'date': 'Thu, 09 Jan 2025 11:11:26 GMT'}, 'RetryAttempts': 0}, 'AcceptRanges': 'bytes', 'LastModified': datetime.datetime(2025, 1, 9, 11, 7, 10, tzinfo=tzutc()), 'ContentLength': 525187, 'ETag': '"a5eeae83d792e

In [33]:
# df = df[['ph','Solids','Turbidity','Potability']]
# df = df[['ph','Solids','Potability']]
df = df[['ph','Potability']]
df.dropna(inplace=True)

In [34]:
mlflow.set_tracking_uri(tracker_url)
experiment_name = "water_potability"
mlflow.set_experiment(experiment_name)

data = df.drop('Potability', axis=1)
target = df['Potability']
X_train, X_test, y_train, y_test = train_test_split(data, target, test_size=0.2, random_state=42)

In [35]:
from mlflow.models.signature import infer_signature
import numpy as np
import pandas as pd
from sklearn.metrics import accuracy_score

# Model parameters
n_estimators = 100
max_depth = 5
random_state = 42

with mlflow.start_run():
    # Log parameters
    mlflow.log_param("n_estimators", n_estimators)
    mlflow.log_param("max_depth", max_depth)
    mlflow.log_param("random_state", random_state)

    # Initialize and train the model
    model = LogisticRegression()
    model.fit(X_train, y_train)

    # Evaluate the model
    predictions = model.predict(X_test)
    accuracy = accuracy_score(y_test, predictions)
    mlflow.log_metric("accuracy", accuracy)

    # Define input example and signature
    input_example = X_test.iloc[:1].fillna(X_test.mean())
    signature = infer_signature(X_test, model.predict(X_test))

    # Log the model with MLflow
    mlflow.sklearn.log_model(
        model,
        artifact_path="model",
        input_example=input_example,
        signature=signature
    )

    # Print run information
    run_id = mlflow.active_run().info.run_id
    print(f"Run ID: {run_id}")
    print(f"Model accuracy: {accuracy}")


2025/01/09 18:11:27 WARNING mlflow.system_metrics.system_metrics_monitor: Skip logging GPU metrics because creating `GPUMonitor` failed with error: `pynvml` is not installed, to log GPU metrics please run `pip install pynvml` to install it..
2025/01/09 18:11:27 INFO mlflow.system_metrics.system_metrics_monitor: Started monitoring system metrics.


Run ID: 109cbf5da48e4d80937ef23189a4511c
Model accuracy: 0.5978456014362658
🏃 View run righteous-rat-478 at: http://magang.fauzanghaza.com/#/experiments/1/runs/109cbf5da48e4d80937ef23189a4511c
🧪 View experiment at: http://magang.fauzanghaza.com/#/experiments/1


2025/01/09 18:11:33 INFO mlflow.system_metrics.system_metrics_monitor: Stopping system metrics monitoring...
2025/01/09 18:11:33 INFO mlflow.system_metrics.system_metrics_monitor: Successfully terminated system metrics monitoring!


In [36]:
from mlflow.models.signature import infer_signature
import numpy as np
import pandas as pd
from sklearn.metrics import accuracy_score

# Model parameters
n_estimators = 100
max_depth = 5
random_state = 42

with mlflow.start_run():
    # Log parameters
    mlflow.log_param("n_estimators", n_estimators)
    mlflow.log_param("max_depth", max_depth)
    mlflow.log_param("random_state", random_state)

    # Initialize and train the model
    model = RandomForestClassifier(n_estimators=n_estimators, max_depth=max_depth, random_state=random_state)
    model.fit(X_train, y_train)

    # Evaluate the model
    predictions = model.predict(X_test)
    accuracy = accuracy_score(y_test, predictions)
    mlflow.log_metric("accuracy", accuracy)

    # Define input example and signature
    input_example = X_test.iloc[:1].fillna(X_test.mean())
    signature = infer_signature(X_test, model.predict(X_test))

    # Log the model with MLflow
    mlflow.sklearn.log_model(
        model,
        artifact_path="model",
        input_example=input_example,
        signature=signature
    )

    # Print run information
    run_id = mlflow.active_run().info.run_id
    print(f"Run ID: {run_id}")
    print(f"Model accuracy: {accuracy}")


2025/01/09 18:11:33 WARNING mlflow.system_metrics.system_metrics_monitor: Skip logging GPU metrics because creating `GPUMonitor` failed with error: `pynvml` is not installed, to log GPU metrics please run `pip install pynvml` to install it..
2025/01/09 18:11:33 INFO mlflow.system_metrics.system_metrics_monitor: Started monitoring system metrics.


Run ID: 8f7a3c2cd19f4e439d79fe3faa3fed5f
Model accuracy: 0.5906642728904847
🏃 View run hilarious-fly-708 at: http://magang.fauzanghaza.com/#/experiments/1/runs/8f7a3c2cd19f4e439d79fe3faa3fed5f
🧪 View experiment at: http://magang.fauzanghaza.com/#/experiments/1


2025/01/09 18:11:39 INFO mlflow.system_metrics.system_metrics_monitor: Stopping system metrics monitoring...
2025/01/09 18:11:40 INFO mlflow.system_metrics.system_metrics_monitor: Successfully terminated system metrics monitoring!


In [37]:
import mlflow.pyfunc

model_uri = f"runs:/{run_id}/model"
model = mlflow.pyfunc.load_model(model_uri)

# Check if metadata and signature are available
if model.metadata:
    print("Model signature:", model.metadata.get_input_schema())
else:
    print("Model metadata is missing.")

Model signature: ['ph': double (required)]
